### Bibliotecas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple

### Parâmetros

In [2]:
# Configuração para replicabilidade
torch.manual_seed(42)
np.random.seed(42)

class GANConfig:
    'Stores all the hyperparameters of the experiment'
    data_dim: int = 2       # Saída do Gerador e a entrada do Discriminador
    noise_dim: int = 1      # Entrada do Gerador
    hidden_dim: int = 128   # Número de neurônios nas camadas ocultas
    batch_size: int = 64    # Número de amostras processadas por iteração de treinamento
    num_epochs: int = 5000  # Número de vezes que o conjunto passa por todo o modelo
    
CASE_STABLE = {
    'name': 'stable',
    'lr_D': 0.00005,
    'lr_G': 0.0001
}

CASE_UNSTABLE = {
    'name': 'unstable',
    'lr_D': 0.00009,
    'lr_G': 0.0001
}

### Distribuições de Dados

`real_data_sampler`: Define a distribuição de dados alvo.

$$y = \sin(x) + \mathcal{N}(0, 0.1)$$

onde $x$ é amostrado uniformemente em $[-3, 3]$. O GAN tentará replicar essa curva senoidal com ruído.

`noise_sampler`: É a entrada do Gerador, amostrada de uma distribuição normal padrão (ruído gaussiano).

$$\mathbf{z} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$$

O Gerador aprende a mapear esse vetor de ruído (`noise_dim`) para a distribuição de dados de saída (`data_dim=2`).

In [3]:
def real_data_sampler(num_samples: int) -> torch.Tensor:
    x = np.random.uniform(-3, 3, size=(num_samples, 1))     # x ~ Uniform(-3, 3)
    noise = np.random.normal(0, 0.1, size=(num_samples, 1)) # Ruído ~ Normal(0, 0.1)
    y = np.sin(x) + noise                                   # y = sin(x) + noise
    data = np.hstack((x, y))
    return torch.tensor(data, dtype=torch.float32)          # Retorna o tensor [x, y]
 
def noise_sampler(num_samples: int) -> torch.Tensor:
    # Retorna num_samples vetores de ruído gaussiano (dimensão noise_dim=1)
    return torch.randn(num_samples, GANConfig.noise_dim)             

### Arquitetura da rede

A rede do Gerador transforma ruído em dados falsos, com a função de ativação ReLU nas camadas ocultas.

Já a rede do Classificador classifica a entrada como real (próximo de 1) ou falsa (próximo de 0), usando LeakyReLU nas camadas ocultas e Sigmoid na saída.

In [4]:
class Generator(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_size: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        return self.net(z)


class Discriminator(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

### Classe de treinamento

A classe `GANTrainer` encapsula todo o ciclo de vida do treinamento de uma GAN.

- Inicialização (`__init__`):
    - Cria instâncias do **Gerador ($\text{G}$)** e do **Discriminador ($\text{D}$)**, que são as duas redes neurais da GAN.
    - Define a função de perda (**`nn.BCELoss()`**, ou **Binary Cross-Entropy Loss**), ideal para tarefas de classificação binária (real vs. fake).
    - Configura os otimizadores (**`optim.Adam`**) separadamente para $\text{G}$ e $\text{D}$, cada um com sua própria taxa de aprendizado ($\text{lr}_{\text{D}}$, $\text{lr}_{\text{G}}$).
    - Prepara o diretório para salvar os **checkpoints** (pesos) do modelo.

O método `train` executa o loop de treinamento por um número fixo de épocas, alternando entre a otimização do Discriminador e a do Gerador em cada passo.

#### D

O objetivo do Discriminador é se tornar bom em distinguir dados reais de dados falsos.

- Amostragem de Dados:
    - Obtém um lote de dados reais (`real_data`) com rótulos $\mathbf{1}$ (`real_labels`).
    - Gera dados falsos (`fake_data`) a partir do $\text{G}$ usando ruído aleatório ($\mathbf{z}$), e os rotula como $\mathbf{0}$ (`fake_labels`).
    - Importante: $\text{G}$ é desconectado do gráfico computacional (`.detach()`) para que os gradientes calculados no $\text{D}$ não sejam propagados de volta para o $\text{G}$ neste passo.
- Cálculo e Otimização:
    - Os dados reais e falsos são concatenados (`all_data`).
    - A $\text{D}$ avalia esses dados (`D_output`).
    - Calcula-se a perda ($\text{loss}_{\text{D}}$) comparando as previsões de $\text{D}$ com os rótulos verdadeiros (1s e 0s).
    - O D é atualizado para minimizar essa perda, ou seja, para melhorar sua capacidade de classificar corretamente os dados.

#### G

O objetivo do Gerador é produzir dados que sejam convincentes o suficiente para enganar o Discriminador.

- Geração de Dados:
    - $\text{G}$ gera um novo lote de dados falsos (`fake_data`) a partir de ruído ($\mathbf{z}$).
    - Define-se o rótulo-alvo como $\mathbf{1}$ (`target_labels`), indicando que o $\text{G}$ está sendo treinado para que o $\text{D}$ classifique sua saída como **real**.
- Cálculo e Otimização:
    - $\text{D}$ avalia os dados gerados (`G_output`).
    - Calcula-se a perda ($\text{loss}_{\text{G}}$) comparando as previsões de $\text{D}$ com o rótulo-alvo **$\mathbf{1}$**.
    - O **Gerador** é atualizado para **minimizar** essa perda, ou seja, para fazer com que o $\text{D}$ atribua uma probabilidade próxima de $\mathbf{1}$ aos dados que ele gera.

Eu adicionei um processo de checkpoints e uma parte de monitoramento. Então a norma do gradiente do $\text{G}$ (`G_grad_norms`) é calculada e armazenada para monitorar a estabilidade do treinamento. A cada 500 épocas, o código imprime as perdas atuais do $\text{D}$ e do $\text{G}$. Em épocas predefinidas (`self.checkpoints`), os estados (pesos) das redes $\text{G}$ e $\text{D}$ são salvos (checkpoint) no disco. O método retorna as listas de perdas ($\text{D}$ e $\text{G}$), as normas dos gradientes de $\text{G}$, as épocas de checkpoint e o diretório de salvamento.

In [ ]:
class GANTrainer:
    'Encapsulates model initialization, the training loop, and checkpoint saving'
    def __init__(self, lr_D: float, lr_G: float, case_name: str):
        self.config = GANConfig()
        self.case_name = case_name
        self.checkpoint_dir = os.path.join('..', 'results', '_1_gan', f'gan_checkpoints_{case_name}')
        os.makedirs(self.checkpoint_dir, exist_ok=True)

        self.G = Generator(self.config.noise_dim, self.config.data_dim, self.config.hidden_dim)
        self.D = Discriminator(self.config.data_dim, self.config.hidden_dim)
        self.criterion = nn.BCELoss()
        self.optimizer_D = optim.Adam(self.D.parameters(), lr=lr_D)
        self.optimizer_G = optim.Adam(self.G.parameters(), lr=lr_G)

        os.makedirs(self.checkpoint_dir, exist_ok=True)
        self.checkpoints: List[int] = [
            int(self.config.num_epochs * 0.25),
            int(self.config.num_epochs * 0.50),
            int(self.config.num_epochs * 0.75),
            self.config.num_epochs
        ]

    def train(self) -> Tuple[List[float], List[float], List[float], List[int], str]:
        D_losses: List[float] = []
        G_losses: List[float] = []
        G_grad_norms: List[float] = []
        
        print(f"\nStarting Training: {self.case_name.upper()}")

        for epoch in range(1, self.config.num_epochs + 1):
            # D
            real_data = real_data_sampler(self.config.batch_size)
            real_labels = torch.ones(self.config.batch_size, 1)
            z = noise_sampler(self.config.batch_size)
            fake_data = self.G(z).detach()
            fake_labels = torch.zeros(self.config.batch_size, 1)
            all_data = torch.cat((real_data, fake_data))
            all_labels = torch.cat((real_labels, fake_labels))
            D_output = self.D(all_data)
            loss_D = self.criterion(D_output, all_labels)
            
            self.D.zero_grad()
            loss_D.backward()
            self.optimizer_D.step()
            D_losses.append(loss_D.item())

            # G
            z = noise_sampler(self.config.batch_size)
            fake_data = self.G(z)
            target_labels = torch.ones(self.config.batch_size, 1)
            G_output = self.D(fake_data)
            loss_G = self.criterion(G_output, target_labels)
            
            self.G.zero_grad()
            loss_G.backward()
            
            # Cálculo e Armazenamento da Norma do Gradiente
            grad_norm = 0.0
            for p in self.G.parameters():
                if p.grad is not None:
                    grad_norm += (p.grad.norm(2).item() ** 2)
            G_grad_norms.append(grad_norm ** 0.5)

            self.optimizer_G.step()
            G_losses.append(loss_G.item())

            if epoch % 500 == 0:
                print(f"Epoch {epoch}/{self.config.num_epochs} | Loss D: {loss_D.item():.4f} | Loss G: {loss_G.item():.4f}")
            
            if epoch in self.checkpoints:
                torch.save(self.G.state_dict(), os.path.join(self.checkpoint_dir, f'generator_epoch_{epoch}.pth'))
                torch.save(self.D.state_dict(), os.path.join(self.checkpoint_dir, f'discriminator_epoch_{epoch}.pth'))
                print(f"Checkpoints saved for epoch {epoch}.")

        return D_losses, G_losses, G_grad_norms, self.checkpoints, self.checkpoint_dir

### Classe de Visualização

Separei alguns gráficos que acho interessante serem usados para avaliar. O objetivo aqui é monitorar o comportamento interno da GAN durante o processo de otimização.

##### `plot_losses()`: Perdas do Discriminador e do Gerador

A finalidade é visualizar o equilíbrio e a convergência do processo adversarial.
Então iremos imprimir a $\text{Loss}_{\text{D}}$ e a $\text{Loss}_{\text{G}}$ ao longo das iterações (epochs).

Um cenário ideal mostra ambas as perdas estabilizando em um valor de equilíbrio.
    - Se a $\text{Loss}_{\text{D}}$ cair para zero, o Discriminador é muito bom e o G não consegue mais aprender (falha de G).
    - Se a $\text{Loss}_{\text{G}}$ cair para zero, o G está enganando facilmente o Discriminador, mas isso não garante a qualidade dos dados (pode indicar mode collapse).
    - Oscilações extremas indicam instabilidade no treinamento.

##### `plot_gradient_norm()`: Norma do Gradiente do Gerador

A finalidade aqui já é monitorar o fluxo de gradiente no G para detectar problemas como vanishing gradient ou exploding gradient.
Nesse caso nós imprimimos a $\text{Norma } \mathbf{L2}$ do gradiente do G ao longo das iterações.
Uma norma de gradiente que se aproxima de zero indica que o G está aprendendo muito pouco ou nada, sugerindo o problema de vanishing gradient.
Normas de gradiente muito altas indicam instabilidade ou a necessidade de weight clipping.

##### `plot_data_evolution()`: Evolução dos Pontos de Dados Gerados

O foco é visualizar diretamente a evolução da capacidade de geração ao longo do treinamento (usando checkpoints). É especialmente útil para dados 2D.
Então temos um gráfico de dispersão (scatter plot) em 2D que compara os dados reais com os dados falsos em diferentes estágios do treinamento.
No início, os pontos falsos estarão dispersos ou muito distantes dos reais. À medida que o treinamento avança, os pontos falsos devem se sobrepor e imitar a forma e a distribuição dos pontos reais, indicando convergência.

##### `plot_diversity_histogram()`: Histograma de Diversidade (Eixo X)

Aqui eu busquei avaliar a diversidade e a cobertura de modo do G. O código usa o eixo X como exemplo.
Compara os histogramas de densidade da coordenada X dos dados Reais e dos dados Falsos em diferentes checkpoints.

Se o histograma dos dados Falsos (vermelho) cobre e se assemelha ao histograma dos dados Reais (azul) em todos os picos e vales da distribuição, o Gerador está capturando bem a diversidade. Se o histograma Falso tiver picos apenas em algumas regiões e ignorar outras (os "modos"), isso indica (mode collapse), um problema comum onde a GAN falha em gerar toda a variedade de dados.

##### `plot_boundary_evolution()`: Evolução da Fronteira de Decisão

O objetivo principal foi mostrar como o D aprende a separar e o Gerador aprende a enganar ao longo do tempo.
A imagem ilustra fronteira de decisão do D (a curva onde a probabilidade de ser real é de $\mathbf{0.5}$) e as regiões classificadas como Real (vermelho/quente) ou Falso (azul/frio). Os dados reais e falsos gerados também são plotados.

No início, a fronteira de decisão (linha preta) pode ser aleatória ou simples. À medida que o treinamento avança, o Discriminador tenta traçar uma fronteira complexa para separar os pontos azuis (reais) dos vermelhos (falsos). No equilíbrio, a fronteira deve se tornar ambígua ou muito complexa, com os dados reais e falsos se **misturando** na região de $\mathbf{0.5}$ (a $\text{D}$ não consegue mais distinguir perfeitamente).

##### `plot_kde_evolution()`: Evolução da Densidade (KDE)

Aqui a ideia era fornecer uma visão suavizada da distribuição de probabilidade dos dados gerados, confirmando a semelhança com a distribuição real.
Então tentei imprimir uma Estimativa de Densidade de Kernel (KDE) para os dados Reais (azul) e Falsos (vermelho) em diferentes checkpoints.

Semelhante ao histograma e ao `plot_data_evolution`, se as áreas de densidade Falsa (vermelho) se sobrepuserem e replicarem fielmente as áreas de densidade Real (azul), a GAN está funcionando bem e o Gerador está capturando a distribuição subjacente dos dados.

In [ ]:
class GANVisualizer:
    'It encapsulates all the logic for plotting and visualizing the results'
    def __init__(self, 
                 D_losses: List[float], 
                 G_losses: List[float], 
                 G_grad_norms: List[float], 
                 checkpoints: List[int], 
                 checkpoint_dir: str, 
                 case_name: str):
        
        self.config = GANConfig()
        self.D_losses = D_losses
        self.G_losses = G_losses
        self.G_grad_norms = G_grad_norms
        self.checkpoints = checkpoints
        self.checkpoint_dir = checkpoint_dir
        self.case_name = case_name
        self.filename_suffix = case_name.lower()
        self.num_epochs = self.config.num_epochs
        
        self.image_output_dir = os.path.join('..', 'results', '_1_gan', 'images')
        os.makedirs(self.image_output_dir, exist_ok=True)

    def _load_model(self, epoch: int) -> Tuple[Generator, Discriminator]:
        'Auxiliary function for loading the G and D of a checkpoint'
        G_checkpoint = Generator(self.config.noise_dim, self.config.data_dim, self.config.hidden_dim)
        D_checkpoint = Discriminator(self.config.data_dim, self.config.hidden_dim)
        
        G_path = os.path.join(self.checkpoint_dir, f'generator_epoch_{epoch}.pth')
        D_path = os.path.join(self.checkpoint_dir, f'discriminator_epoch_{epoch}.pth') 
        
        if not os.path.exists(G_path) or not os.path.exists(D_path):
            raise FileNotFoundError(f"Checkpoints not found for epoch {epoch}.")

        G_checkpoint.load_state_dict(torch.load(G_path))
        D_checkpoint.load_state_dict(torch.load(D_path))
        G_checkpoint.eval()
        D_checkpoint.eval()
        return G_checkpoint, D_checkpoint

    def plot_losses(self) -> str:
        plt.figure(figsize=(10, 5)) 
        plt.plot(self.D_losses, label='Discriminator Loss', color='blue')
        plt.plot(self.G_losses, label='Generator Loss', color='red')
        plt.title(f'GAN Training Losses ({self.case_name.upper()})') 
        plt.xlabel('Iteration') 
        plt.ylabel('Loss') 
        plt.legend() 
        plt.grid(True) 
        filename = f'gan_losses_evolution_{self.filename_suffix}.png'
        full_path = os.path.join(self.image_output_dir, filename)
        plt.savefig(full_path) 
        plt.close()
        return filename

    def plot_gradient_norm(self) -> str:
        plt.figure(figsize=(10, 5)) 
        plt.plot(self.G_grad_norms, label='Generator Gradient L2 Norm', color='green')
        plt.title(f'Generator Gradient Norm Evolution ({self.case_name.upper()} - Vanishing Gradient)') 
        plt.xlabel('Iteration') 
        plt.ylabel('Gradient L2 Norm') 
        plt.legend() 
        plt.grid(True) 
        filename = f'gan_grad_norm_evolution_{self.filename_suffix}.png'
        full_path = os.path.join(self.image_output_dir, filename)
        plt.savefig(full_path)  
        plt.close()
        return filename

    def plot_data_evolution(self) -> str:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        num_samples = 1000
        real_data = real_data_sampler(num_samples).numpy()
        
        for i, epoch in enumerate(self.checkpoints):
            try:
                G_checkpoint, _ = self._load_model(epoch)
            except FileNotFoundError:
                continue

            z = noise_sampler(num_samples)
            fake_data = G_checkpoint(z).detach().numpy()
            
            ax = axes[i]
            ax.scatter(real_data[:, 0], real_data[:, 1], s=5, alpha=0.6, label='Real data', color='blue')
            ax.scatter(fake_data[:, 0], fake_data[:, 1], s=5, alpha=0.6, color='red', label='Fake data')
            
            percent = int(epoch / self.num_epochs * 100)
            ax.set_title(f'{self.case_name.upper()} Data Evolution - {percent}% ({epoch} Epochs)')
            ax.set_xlabel('x')
            ax.set_ylabel('y')
            ax.legend()
            ax.grid(True)

        plt.tight_layout()
        filename = f'gan_data_evolution_{self.filename_suffix}.png'
        full_path = os.path.join(self.image_output_dir, filename)
        plt.savefig(full_path) 
        plt.close()
        return filename

    def plot_diversity_histogram(self) -> str:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        num_samples = 5000 
        real_data_x = real_data_sampler(num_samples).numpy()[:, 0]
        x_bins = np.linspace(-3.5, 3.5, 50) 
        
        for i, epoch in enumerate(self.checkpoints):
            try:
                G_checkpoint, _ = self._load_model(epoch)
            except FileNotFoundError:
                continue

            z = noise_sampler(num_samples)
            fake_data_x = G_checkpoint(z).detach().numpy()[:, 0]
            
            ax = axes[i]
            ax.hist(real_data_x, bins=x_bins, density=True, alpha=0.5, color='blue', label='Real Data (Target Uniform)')
            ax.hist(fake_data_x, bins=x_bins, density=True, alpha=0.7, color='red', label='Fake Data (Generated)')
            
            percent = int(epoch / self.num_epochs * 100)
            ax.set_title(f'{self.case_name.upper()} X-Diversity - {percent}% ({epoch} Epochs)')
            ax.set_xlabel('X Coordinate')
            ax.set_ylabel('Density')
            ax.legend()
            ax.grid(axis='y')

        plt.tight_layout()
        filename = f'gan_x_diversity_histogram_{self.filename_suffix}.png'
        full_path = os.path.join(self.image_output_dir, filename)
        plt.savefig(full_path) 
        plt.close()
        return filename

    def plot_boundary_evolution(self) -> str:
        fig, axes = plt.subplots(2, 2, figsize=(14, 14))
        axes = axes.flatten()
        num_samples = 1000
        real_data = real_data_sampler(num_samples).numpy()
        
        for i, epoch in enumerate(self.checkpoints):
            try:
                G_checkpoint, D_checkpoint = self._load_model(epoch)
            except FileNotFoundError:
                print(f"Skipping epoch {epoch}: Checkpoint files not found.")
                continue

            self._plot_decision_boundary(G_checkpoint, D_checkpoint, axes[i], epoch, self.num_epochs, real_data, self.case_name)

        plt.tight_layout()
        filename = f'gan_boundary_evolution_{self.filename_suffix}.png'
        full_path = os.path.join(self.image_output_dir, filename)
        plt.savefig(full_path) 
        plt.close()
        return filename
    
    def _plot_decision_boundary(self, G_checkpoint, D_checkpoint, ax, epoch, num_epochs, real_data, case_name):
        x_min, x_max = -4, 4
        y_min, y_max = -2, 2
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                            np.arange(y_min, y_max, 0.05))
        
        grid_points = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
        
        with torch.no_grad():
            Z = D_checkpoint(grid_points).numpy().reshape(xx.shape)

        ax.contourf(xx, yy, Z, levels=np.linspace(0, 1, 11), cmap=plt.cm.RdBu, alpha=0.4)
        ax.contour(xx, yy, Z, levels=[0.5], linewidths=2, colors='k')

        num_samples = 1000
        z = noise_sampler(num_samples)
        fake_data = G_checkpoint(z).detach().numpy()

        ax.scatter(real_data[:, 0], real_data[:, 1], s=5, alpha=0.6, label='Real Data', color='blue')
        ax.scatter(fake_data[:, 0], fake_data[:, 1], s=5, alpha=0.6, color='red', label='Fake Data (Geração)')

        percent = int(epoch / num_epochs * 100)
        ax.set_title(f'{case_name.upper()} Boundary - {percent}% ({epoch} Epochs)')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.legend()
        ax.grid(True)
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)

    def plot_kde_evolution(self) -> str:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        num_samples = 1000
        real_data = real_data_sampler(num_samples).numpy()
        
        for i, epoch in enumerate(self.checkpoints):
            try:
                G_checkpoint, _ = self._load_model(epoch)
            except FileNotFoundError:
                continue

            z = noise_sampler(num_samples)
            fake_data = G_checkpoint(z).detach().numpy()
            
            ax = axes[i]
            sns.kdeplot(x=real_data[:, 0], y=real_data[:, 1], ax=ax, cmap="Blues", fill=True, alpha=0.5, label='Real Data Density')
            sns.kdeplot(x=fake_data[:, 0], y=fake_data[:, 1], ax=ax, cmap="Reds", fill=True, alpha=0.5, label='Fake Data Density')

            percent = int(epoch / self.num_epochs * 100)
            ax.set_title(f'{self.case_name.upper()} KDE Density - {percent}% ({epoch} Epochs)')
            ax.set_xlabel('x')
            ax.set_ylabel('y')
            ax.set_xlim(-4, 4)
            ax.set_ylim(-2, 2)

        plt.tight_layout()
        filename = f'gan_kde_evolution_{self.filename_suffix}.png'
        full_path = os.path.join(self.image_output_dir, filename)
        plt.savefig(full_path) 
        plt.close()
        return filename

    def generate_all_plots(self):
        'Public method for generating all graphs'
        self.plot_losses()
        self.plot_gradient_norm()
        self.plot_data_evolution()
        self.plot_diversity_histogram()
        self.plot_boundary_evolution()
        self.plot_kde_evolution()

### Start

Uma pergunta que pode surgir: **Qual motivo do learning_rate sempre ser menor no D do que no G?**

É uma heurística de treinamento comum para ajudar a estabilizar o processo de treinamento devido à natureza de jogo de soma zero.

O D tem o objetivo de distinguir dados reais (da distribuição $y = \sin(x) + \text{noise}$) de dados falsos gerados. Se o D for treinado muito rapidamente, ele pode se tornar "muito bom, muito rápido". Isso significa que a sua saída para os dados falsos será sempre muito próxima de 0, e para os dados reais, muito próxima de 1. 

Nessa situação, o G receberia gradientes muito pequenos e de baixa qualidade (ou muito fortes, *vanishing/exploding gradients*) do D, tornando extremamente difícil para ele aprender a melhorar. O G pode ficar estagnado porque o sinal de erro (gradiente) do D é muito fraco ou inconsistente. A ideia é que o G deve estar sempre "um pouco à frente" para forçar o D a continuar a melhorar sua capacidade de detecção. 

Então precisamos atrasar intencionalmente o aprendizado de D (não sobrou nada para o beta...) para garantir que G tenha uma chance de acompanhar e que o treinamento não entre em colapso.

Se $LR_D \ge LR_G$, D pode ganhar o jogo rapidamente e reportar consistentemente que as amostras de G são falsas, o que leva ao problema de Vanishing Gradient, Mode Collapse ou à falha total do treinamento.

A proporção $LR_D = \frac{1}{2} LR_G$ é uma configuração **empírica**.

In [7]:
print("Start!")

# Caso Estável
trainer_s = GANTrainer(
    lr_D=CASE_STABLE['lr_D'], 
    lr_G=CASE_STABLE['lr_G'], 
    case_name=CASE_STABLE['name']
)
D_losses_s, G_losses_s, G_grad_norms_s, checkpoints_s, dir_s = trainer_s.train()

visualizer_s = GANVisualizer(
    D_losses_s, G_losses_s, G_grad_norms_s, checkpoints_s, dir_s, CASE_STABLE['name']
)
visualizer_s.generate_all_plots()

# Caso Instável
trainer_u = GANTrainer(
    lr_D=CASE_UNSTABLE['lr_D'], 
    lr_G=CASE_UNSTABLE['lr_G'], 
    case_name=CASE_UNSTABLE['name']
)
D_losses_u, G_losses_u, G_grad_norms_u, checkpoints_u, dir_u = trainer_u.train()

visualizer_u = GANVisualizer(
    D_losses_u, G_losses_u, G_grad_norms_u, checkpoints_u, dir_u, CASE_UNSTABLE['name']
)
visualizer_u.generate_all_plots()

print("Completed!")

Start!

Iniciando Treinamento: STABLE
Epoch 500/5000 | Loss D: 0.5182 | Loss G: 1.0186
Epoch 1000/5000 | Loss D: 0.7174 | Loss G: 0.8638
Checkpoints saved for epoch 1250.
Epoch 1500/5000 | Loss D: 0.6925 | Loss G: 0.6715
Epoch 2000/5000 | Loss D: 0.6871 | Loss G: 0.6907
Epoch 2500/5000 | Loss D: 0.6946 | Loss G: 0.6887
Checkpoints saved for epoch 2500.
Epoch 3000/5000 | Loss D: 0.6856 | Loss G: 0.7092
Epoch 3500/5000 | Loss D: 0.6958 | Loss G: 0.6848
Checkpoints saved for epoch 3750.
Epoch 4000/5000 | Loss D: 0.6916 | Loss G: 0.6873
Epoch 4500/5000 | Loss D: 0.6937 | Loss G: 0.6698
Epoch 5000/5000 | Loss D: 0.6879 | Loss G: 0.6883
Checkpoints saved for epoch 5000.

Iniciando Treinamento: UNSTABLE
Epoch 500/5000 | Loss D: 0.7012 | Loss G: 0.6593
Epoch 1000/5000 | Loss D: 0.6929 | Loss G: 0.6949
Checkpoints saved for epoch 1250.
Epoch 1500/5000 | Loss D: 0.6799 | Loss G: 0.7171
Epoch 2000/5000 | Loss D: 0.6904 | Loss G: 0.7028
Epoch 2500/5000 | Loss D: 0.6956 | Loss G: 0.7074
Checkpoints

# Análise dos resultados

## Perdas

Caso estável: Treinamento ideal. As perdas (D e G) convergem e se estabilizam rapidamente em torno de 0.69 (≈-ln(0.5)), indicando que D e G atingiram um equilíbrio onde D classifica as amostras falsas com probabilidade de 50%

Caso instável: Treinamento volátil. As perdas oscilam drasticamente no início e continuam com variações de alta frequência ao longo de todo o treinamento, indicando que o equilíbrio é frágil e as redes estão se sobrepondo constantemente

<div style="display: flex;">
    <img src="../results/_1_gan/images/gan_losses_evolution_stable.png" alt="Gráfico de Perdas do GAN - Imagem 1" style="width: 50%; padding-right: 5px;" />
    <img src="../results/_1_gan/images/gan_losses_evolution_unstable.png" alt="Gráfico de Perdas do GAN - Imagem 2" style="width: 50%; padding-left: 5px;" />
</div>

## Norma do Gradiente

Caso estável: Após uma fase inicial turbulenta, a norma do gradiente de G diminui e se estabiliza em torno de zero, mas com pequenas oscilações. Sugere que o G está recebendo gradientes mais consistentes (estabilidade)

Caso instável: Maior volatilidade. A norma do gradiente de G apresenta picos recorrentes e uma oscilação geral maior do que no cenário estável. Isso sugere que a G tem dificuldade em receber feedback consistente

<div style="display: flex;">
    <img src="../results/_1_gan/images/gan_grad_norm_evolution_stable.png" alt="Gráfico de Perdas do GAN - Imagem 1" style="width: 50%; padding-right: 5px;" />
    <img src="../results/_1_gan/images/gan_grad_norm_evolution_unstable.png" alt="Gráfico de Perdas do GAN - Imagem 2" style="width: 50%; padding-left: 5px;" />
</div>

## Evolução dos Dados

Caso estável: Excelente convergência. O Gerador (pontos vermelhos) imita com precisão a forma da distribuição real (pontos azuis) a partir de 75% do treinamento.

Caso instável: Geração imperfeita. Embora a G aprenda a forma básica, há pontos falsos mais dispersos nas extremidades, e o Gerador não cobre totalmente o domínio dos dados reais, indicando uma falha menor na cobertura de modo ou qualidade

<div style="display: flex;">
    <img src="../results/_1_gan/images/gan_data_evolution_stable.png" alt="Gráfico de Perdas do GAN - Imagem 1" style="width: 50%; padding-right: 5px;" />
    <img src="../results/_1_gan/images/gan_data_evolution_unstable.png" alt="Gráfico de Perdas do GAN - Imagem 2" style="width: 50%; padding-left: 5px;" />
</div>

## Fronteira

Caso estável: Fronteira adaptável. O Discriminador cria uma fronteira de decisão complexa (linha preta) que se adapta perfeitamente à distribuição real. Na época final (100%), a região classificada como ""real"" (azul claro) envolve a curva de dados, indicando que G está gerando dados indistinguíveis

Caso instável: Fronteira ""tentativa-e-erro"". A linha de decisão (preta) é extremamente irregular e instável, indicando que o Discriminador nunca se acalma e está constantemente aprendendo e esquecendo a fronteira correta devido à natureza volátil dos dados gerados

<div style="display: flex;">
    <img src="../results/_1_gan/images/gan_boundary_evolution_stable.png" alt="Gráfico de Perdas do GAN - Imagem 1" style="width: 50%; padding-right: 5px;" />
    <img src="../results/_1_gan/images/gan_boundary_evolution_unstable.png" alt="Gráfico de Perdas do GAN - Imagem 2" style="width: 50%; padding-left: 5px;" />
</div>

## KDE

Caso estável: Distribuições alinhadas. A densidade dos dados gerados (vermelho) cobre e replica a densidade dos dados reais (azul), confirmando que a distribuição de probabilidade foi aprendida

Caso instável: Dificuldade na Convergência. A sobreposição das densidades é boa, mas em 75% e 100%, a densidade do Gerador (vermelho) parece menos suave e ligeiramente deslocada da densidade real (azul) na extremidade esquerda, refletindo a instabilidade e o ruído

<div style="display: flex;">
    <img src="../results/_1_gan/images/gan_kde_evolution_stable.png" alt="Gráfico de Perdas do GAN - Imagem 1" style="width: 50%; padding-right: 5px;" />
    <img src="../results/_1_gan/images/gan_kde_evolution_unstable.png" alt="Gráfico de Perdas do GAN - Imagem 2" style="width: 50%; padding-left: 5px;" />
</div>